In [ ]:
import os
import pandas as pd

# Paths to directories
daily_volume_dir = "/content/drive/MyDrive/FYP/cboe_options/cboe_options_filtered"
adv_dir = "/content/drive/MyDrive/FYP/cboe_options/cboe_adv_filtered"
output_dir = "/content/drive/MyDrive/FYP/cboe_options/cboe_options_with_adv"
os.makedirs(output_dir, exist_ok=True)

# Process each year
for daily_file in os.listdir(daily_volume_dir):
    if daily_file.startswith("filtered_daily_volume_") and daily_file.endswith(".csv"):
        year = daily_file.split("_")[-1].replace(".csv", "")  # Extract year
        print(f"Processing year: {year}")

        # Load the daily volume and ADV files
        daily_path = os.path.join(daily_volume_dir, daily_file)
        daily_df = pd.read_csv(daily_path)

        adv_file = f"filtered_monthly_volume_{year}.csv"
        adv_path = os.path.join(adv_dir, adv_file)
        adv_df = pd.read_csv(adv_path)

        # Extract year and month from 'Trade Date' in daily data
        daily_df['Trade Month'] = daily_df['Trade Date'].str[:7]  # Keep only year and month (e.g., "2024/01")

        # Merge ADV data with daily volume data
        merged_df = pd.merge(
            daily_df,
            adv_df.rename(columns={"Trade Month": "Trade Month"}),  # Ensure columns match
            how="left",
            on=["Trade Month", "Options Class"]
        )

        # Save the output for the year
        output_file = os.path.join(output_dir, f"daily_volume_with_adv_{year}.csv")
        merged_df.to_csv(output_file, index=False)
        print(f"Saved combined data for {year} to {output_file}")

In [ ]:
import pandas as pd
import os

# Specify the directory containing the CSV files
directory_path = "/content/drive/MyDrive/FYP/cboe_options/cboe_options_with_adv"

# Specify the output file name
output_file = "/content/drive/MyDrive/FYP/merged_cboe_options.csv"

# List to hold dataframes
dataframes = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(directory_path, filename)
        print(f"Processing: {filename}")

        # Read the CSV file
        df = pd.read_csv(file_path)

        # Drop the specified columns if they exist
        columns_to_drop = ['Trade Month', 'Underlying_y', 'Product Type_y', 'Exchange_y']
        df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], errors='ignore')

        # Append the dataframe to the list
        dataframes.append(df)

# Concatenate all dataframes into a single dataframe
merged_df = pd.concat(dataframes, ignore_index=True)

# Save the merged dataframe to a new CSV file
merged_df.to_csv(output_file, index=False)
print(f"Merged CSV saved to {output_file}")


In [ ]:
import pandas as pd

# Load the merged CSV file
merged_file_path = "/content/drive/MyDrive/FYP/merged_cboe_options.csv"
output_file_path = "/content/drive/MyDrive/FYP/cleaned_merged_cboe_options.csv"

# Read the merged file
df = pd.read_csv(merged_file_path)

# 1. Rename columns to remove the `_x` suffix
df.rename(columns={
    'Underlying_x': 'Underlying',
    'Product Type_x': 'Product Type',
    'Exchange_x': 'Exchange'
}, inplace=True)

# 2. Drop rows where `Underlying` is not equal to `Options Class`
df = df[df['Underlying'] == df['Options Class']]

# 3. Add a `% Spike` column to calculate the percentage difference
# Formula: % Spike = ((Volume - Average Daily Volume) / Average Daily Volume) * 100
df['% Spike'] = ((df['Volume'] - df['Average Daily Volume']) / df['Average Daily Volume']) * 100

# Save the cleaned file
df.to_csv(output_file_path, index=False)

print(f"Cleaned and updated data saved to {output_file_path}")


In [ ]:
import pandas as pd

# File paths
cboe_file_path = "/content/drive/MyDrive/FYP/cleaned_merged_cboe_options.csv"
articles_file_path = "/content/drive/MyDrive/FYP/articles_with_reddit_sentiment_all.csv"
output_file_path = "/content/drive/MyDrive/FYP/merged_articles_with_reddit_with_options_data.csv"

# Load the CSV files
cboe_df = pd.read_csv(cboe_file_path)
articles_df = pd.read_csv(articles_file_path)


# 1. Rename columns in the CBOE dataset
cboe_df.rename(columns={
    'Volume': 'Options Volume',
    'Average Daily Volume': 'Options Average Daily Volume',
    '% Spike': 'Options % Spike'
}, inplace=True)

# 2. Convert date formats to the same standard
cboe_df['Trade Date'] = pd.to_datetime(cboe_df['Trade Date'], format='%Y/%m/%d')
articles_df['Date'] = pd.to_datetime(articles_df['Date'], format='%Y-%m-%d')
print("trade date structure", cboe_df['Trade Date'])
print("article date structure", articles_df['Date'])

# 3. Perform the merge on 'Trade Date' -> 'Date' and 'Underlying' -> 'ticker'
merged_df = pd.merge(
    articles_df,
    cboe_df,
    left_on=['Date', 'ticker'],
    right_on=['Trade Date', 'Underlying'],
    how='left'
)

# 4. Drop unnecessary columns added during the merge
merged_df.drop(columns=['Trade Date', 'Underlying', 'Options Class', 'Product Type', 'Exchange'], inplace=True)


# 5. Save the final merged dataset
merged_df.to_csv(output_file_path, index=False)

print(f"Merged dataset saved to {output_file_path}")

Comparing dates where articles were published to dates where they weren't

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# File paths
csv1_path = "/content/drive/MyDrive/FYP/merged_articles_with_reddit_with_options_data.csv"
csv2_path = "/content/drive/MyDrive/FYP/missing_dates_merged_articles_with_options_data.csv"

# Load the datasets
csv1_df = pd.read_csv(csv1_path)
csv2_df = pd.read_csv(csv2_path)

# Ensure the 'Options % Spike' column is numeric
csv1_df['Options % Spike'] = pd.to_numeric(csv1_df['Options % Spike'].abs(), errors='coerce')
csv2_df['Options % Spike'] = pd.to_numeric(csv2_df['Options % Spike'].abs(), errors='coerce')

# Drop rows with NaN values in the 'Options % Spike' column
csv1_df = csv1_df.dropna(subset=['Options % Spike'])
csv2_df = csv2_df.dropna(subset=['Options % Spike'])

# Add a column to indicate the source
csv1_df['Source'] = 'Normal Dates'
csv2_df['Source'] = 'Missing Dates'

# Combine the two datasets and reset the index
combined_df = pd.concat([csv1_df[['Options % Spike', 'Source']], csv2_df[['Options % Spike', 'Source']]]).reset_index(drop=True)

# Plot the distributions
plt.figure(figsize=(12, 6))
sns.kdeplot(data=combined_df, x='Options % Spike', hue='Source', fill=True, common_norm=False, alpha=0.5)
plt.title("Comparison of Options % Spike for Normal vs Missing Dates")
plt.xlabel("Options % Spike")
plt.ylabel("Density")
plt.grid()
plt.show()

# Boxplot for comparison
plt.figure(figsize=(8, 6))
sns.boxplot(data=combined_df, x='Source', y='Options % Spike')
plt.title("Boxplot of Options % Spike for Normal vs Missing Dates")
plt.xlabel("Source")
plt.ylabel("Options % Spike")
plt.grid()
plt.show()


In [ ]:
# Combine the two datasets and reset the index
combined_df = pd.concat([csv1_df[['Options % Spike', 'Source']], csv2_df[['Options % Spike', 'Source']]]).reset_index(drop=True)

# Generate summary statistics
summary_stats = combined_df.groupby('Source')['Options % Spike'].describe()

# Display summary statistics
print("Summary Statistics for Options % Spike:")
print(summary_stats)

In [ ]:
# Exclude rows where 'Options % Spike' is infinite
combined_df = combined_df[~combined_df['Options % Spike'].isin([float('inf'), float('-inf')])]

# Recalculate summary statistics
filtered_summary_stats = combined_df.groupby('Source')['Options % Spike'].describe()

# Display filtered summary statistics
print("Filtered Summary Statistics for Options % Spike:")
print(filtered_summary_stats)

# Compare means
mean_missing = filtered_summary_stats.loc['Missing Dates', 'mean']
mean_normal = filtered_summary_stats.loc['Normal Dates', 'mean']

if mean_missing > mean_normal:
    print(f"The average spike is greater for missing dates ({mean_missing:.2f}) than for normal dates ({mean_normal:.2f}).")
else:
    print(f"The average spike is greater for normal dates ({mean_normal:.2f}) than for missing dates ({mean_missing:.2f}).")


In [ ]:
import pandas as pd

# File path for the dataset
articles_file_path = "/content/drive/MyDrive/FYP/merged_articles_with_reddit_with_options_data.csv"

# Load the dataset
articles_df = pd.read_csv(articles_file_path)

# Ensure relevant columns are in the correct numeric format
articles_df['% Spike'] = pd.to_numeric(articles_df['% Spike'].abs(), errors='coerce')
articles_df['reddit vader sentiment'] = pd.to_numeric(articles_df['reddit vader sentiment'].abs(), errors='coerce')
articles_df['Options % Spike'] = pd.to_numeric(articles_df['Options % Spike'].abs(), errors='coerce')
articles_df['Sentiment_Score'] = pd.to_numeric(articles_df['Sentiment_Score'].abs(), errors='coerce')

# Drop rows with NaN values in the relevant columns
correlation_df = articles_df[['Sentiment_Score', '% Spike', 'reddit vader sentiment', 'Options % Spike']].dropna()

# Calculate correlations
correlations = correlation_df.corr()

# Display the correlations between Sentiment_Score and the other columns
print("Correlation of Sentiment_Score with other metrics:")
print(correlations['Sentiment_Score'][['% Spike', 'reddit vader sentiment', 'Options % Spike']])


In [ ]:
import pandas as pd

# File path for the dataset
articles_file_path = "/content/drive/MyDrive/FYP/merged_articles_with_reddit_with_options_data.csv"

# Load the dataset
articles_df = pd.read_csv(articles_file_path)

# Drop all rows with any NaN values
articles_df_cleaned = articles_df.dropna()

# Display the number of rows before and after cleaning
print(f"Number of rows before cleaning: {len(articles_df)}")
print(f"Number of rows after cleaning: {len(articles_df_cleaned)}")

# Save the cleaned DataFrame if needed
output_file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"
articles_df_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned dataset saved to: {output_file_path}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# File path for the dataset
file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"

# Load the dataset
articles_df = pd.read_csv(file_path)

# Feature (Sentiment_Score) and target (Options % Spike)
X = articles_df[['Sentiment_Score']]
y = articles_df['Options % Spike']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.scatter(X_test, y_test, color='blue', label='Actual')
plt.scatter(X_test, y_pred, color='red', alpha=0.6, label='Predicted')
plt.title('Linear Regression: Predicting Options % Spike')
plt.xlabel('Sentiment Score')
plt.ylabel('Options % Spike')
plt.legend()
plt.grid()
plt.show()

# Save the model if needed
import joblib
model_file_path = "/content/drive/MyDrive/FYP/options_spike_prediction_model.pkl"
joblib.dump(model, model_file_path)
print(f"Model saved to: {model_file_path}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
data_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"
data = pd.read_csv(data_path)

# Drop rows with missing values
data = data.dropna()

# Use absolute values for the target variable
data['Options % Spike'] = data['Options % Spike'].abs()

# Define features and target variable
X = data[['Sentiment_Score']]  # Predictor: Sentiment Score
y = data['Options % Spike']    # Target: Absolute Options % Spike

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate evaluation metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Visualize actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2, label="Ideal Fit")
plt.title("Linear Regression: Actual vs Predicted Options % Spike")
plt.xlabel("Actual Absolute Options % Spike")
plt.ylabel("Predicted Absolute Options % Spike")
plt.legend()
plt.grid()
plt.show()

# Display the regression coefficient and intercept
print(f"Regression Coefficient (Slope): {model.coef_[0]}")
print(f"Regression Intercept: {model.intercept_}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv("/content/drive/MyDrive/FYP/cleaned_final.csv")

# Preprocessing: Drop rows with missing values and use absolute values
data = data.dropna()
data['Options % Spike'] = data['Options % Spike'].abs()

# Define features (X) and target (y)
X = data[['Sentiment_Score']]
y = data['Options % Spike']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Visualize the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', lw=2)
plt.xlabel("Actual Options % Spike")
plt.ylabel("Predicted Options % Spike")
plt.title("Actual vs Predicted Options % Spike (Random Forest)")
plt.grid()
plt.show()


In [ ]:
from sklearn.neural_network import MLPRegressor

# Initialize Neural Network Regressor
mlp = MLPRegressor(hidden_layer_sizes=(100, 50, 25), max_iter=1000, random_state=42)

# Fit the model on the training data
mlp.fit(X_train, y_train)

# Predict on the test data
y_pred_mlp = mlp.predict(X_test)

# Evaluate the model
mse_mlp = mean_squared_error(y_test, y_pred_mlp)
r2_mlp = r2_score(y_test, y_pred_mlp)

print(f"Neural Network Regressor - Mean Squared Error: {mse_mlp}")
print(f"Neural Network Regressor - R-squared: {r2_mlp}")

# Plot Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_mlp, alpha=0.5, edgecolors="k")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("Actual vs Predicted Options % Spike (Neural Network Regressor)")
plt.xlabel("Actual Options % Spike")
plt.ylabel("Predicted Options % Spike")
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load data
file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"
df = pd.read_csv(file_path)

# Drop rows with missing values
df = df.dropna(subset=['Title', 'Options % Spike'])

# Preprocessing: Target variable and feature
y = df['Options % Spike']  # Use absolute values of % Spike
X = df['Title']

# Convert text to numerical data using TF-IDF
vectorizer = TfidfVectorizer(max_features=1000)  # Limit to top 1000 features
X_tfidf = vectorizer.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Train Gradient Boosting Regressor
gbr = GradientBoostingRegressor(n_estimators=500, learning_rate=0.05, max_depth=5, random_state=42)
gbr.fit(X_train, y_train)

# Predict on the test set
y_pred = gbr.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Plot Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5, edgecolors="k")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("Actual vs Predicted Options % Spike (Gradient Boosting Regressor)")
plt.xlabel("Actual Options % Spike")
plt.ylabel("Predicted Options % Spike")
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load the data
file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"
df = pd.read_csv(file_path)

# Drop rows with missing values
df = df.dropna(subset=['Title', 'Options % Spike'])

# Use absolute values of `Options % Spike` as target
df['Options % Spike'] = df['Options % Spike'].abs()

# Text data: Title (input) and Options % Spike (target)
texts = df['Title'].values
target = df['Options % Spike'].values

# Text Preprocessing
max_words = 10000  # Maximum number of words in the vocabulary
max_len = 100  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

X = pad_sequences(sequences, maxlen=max_len)
y = target

# Scale the target variable
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y.reshape(-1, 1))

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y_scaled[:train_size], y_scaled[train_size:]

# Build the LSTM model
embedding_dim = 100
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, return_sequences=True, activation='tanh'))
model.add(Dropout(0.5))
model.add(LSTM(100, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='linear'))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
batch_size = 64
epochs = 10
history = model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,
    verbose=1
)

# Evaluate the model
y_pred_scaled = model.predict(X_test)
y_pred = scaler.inverse_transform(y_pred_scaled)

# Calculate performance metrics
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_scaled))
print(f"Test RMSE: {test_rmse}")

# Plot Actual vs Predicted
plt.figure(figsize=(10, 6))
plt.scatter(scaler.inverse_transform(y_test), y_pred, alpha=0.5, edgecolors="k")
plt.plot(
    [scaler.inverse_transform(y_test).min(), scaler.inverse_transform(y_test).max()],
    [scaler.inverse_transform(y_test).min(), scaler.inverse_transform(y_test).max()],
    'r--',
    lw=2
)
plt.title("Actual vs Predicted Options % Spike (LSTM)")
plt.xlabel("Actual Options % Spike")
plt.ylabel("Predicted Options % Spike")
plt.grid(True)
plt.show()

# Plot Training Loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"  # Update this to your file's path
data = pd.read_csv(file_path)

# Drop rows with missing values to ensure valid correlation calculations
data = data.dropna(subset=['Sentiment_Score', 'Options % Spike'])

# Group by company and calculate correlation
correlations = data.groupby('Company').apply(
    lambda group: group['Sentiment_Score'].corr(group['Options % Spike'])
)

# Find the company with the highest correlation
highest_correlation = correlations.idxmax()
max_correlation_value = correlations.max()

print(f"The company with the greatest correlation is: {highest_correlation}")
print(f"Correlation value: {max_correlation_value}")

# Display all correlations for context
print("\nAll companies with their correlation values:")
print(correlations.sort_values(ascending=False))


In [ ]:
import pandas as pd

# Load the dataset
file_path = "/content/drive/MyDrive/FYP/cleaned_final.csv"  # Update with your dataset path
df = pd.read_csv(file_path)

# Ensure numerical columns are in the correct format
df['Sentiment_Score'] = pd.to_numeric(df['Sentiment_Score'], errors='coerce')
df['% Spike'] = pd.to_numeric(df['% Spike'], errors='coerce')
df['reddit vader sentiment'] = pd.to_numeric(df['reddit vader sentiment'], errors='coerce')
df['Options % Spike'] = pd.to_numeric(df['Options % Spike'], errors='coerce')

# Drop rows with missing values
df.dropna(subset=['Sentiment_Score', '% Spike', 'reddit vader sentiment', 'Options % Spike'], inplace=True)

# Group by company
company_groups = df.groupby('Company')

# Calculate correlations for each company
correlation_results = []
for company, group in company_groups:
    sentiment_spike_corr = group['Sentiment_Score'].corr(group['% Spike'])
    sentiment_reddit_corr = group['Sentiment_Score'].corr(group['reddit vader sentiment'])
    sentiment_options_spike_corr = group['Sentiment_Score'].corr(group['Options % Spike'])

    correlation_results.append({
        'Company': company,
        'Sentiment_Score vs % Spike': sentiment_spike_corr,
        'Sentiment_Score vs reddit vader sentiment': sentiment_reddit_corr,
        'Sentiment_Score vs Options % Spike': sentiment_options_spike_corr
    })

# Create a DataFrame to store the correlation results
correlation_df = pd.DataFrame(correlation_results)

# Find top performers for each correlation
top_sentiment_spike = correlation_df.sort_values(by='Sentiment_Score vs % Spike', ascending=False).head(5)
top_sentiment_reddit = correlation_df.sort_values(by='Sentiment_Score vs reddit vader sentiment', ascending=False).head(5)
top_sentiment_options_spike = correlation_df.sort_values(by='Sentiment_Score vs Options % Spike', ascending=False).head(5)

# Save the correlation results
correlation_df.to_csv("company_correlations.csv", index=False)

# Print the results
print("Top Companies - Sentiment_Score vs % Spike:")
print(top_sentiment_spike)
print("\nTop Companies - Sentiment_Score vs reddit vader sentiment:")
print(top_sentiment_reddit)
print("\nTop Companies - Sentiment_Score vs Options % Spike:")
print(top_sentiment_options_spike)


In [ ]:

# Ensure numerical columns are in the correct format
df['Sentiment_Score'] = pd.to_numeric(df['Sentiment_Score'], errors='coerce')
df['% Spike'] = pd.to_numeric(df['% Spike'], errors='coerce')
df['reddit vader sentiment'] = pd.to_numeric(df['reddit vader sentiment'], errors='coerce')
df['Options % Spike'] = pd.to_numeric(df['Options % Spike'], errors='coerce')

# Drop rows with missing values
df.dropna(subset=['Sentiment_Score', '% Spike', 'reddit vader sentiment', 'Options % Spike'], inplace=True)

# Group by company
company_groups = df.groupby('Company')

# Calculate correlations for each company
correlation_results = []
for company, group in company_groups:
    sentiment_spike_corr = group['Sentiment_Score'].corr(group['% Spike'])
    sentiment_reddit_corr = group['Sentiment_Score'].corr(group['reddit vader sentiment'])
    sentiment_options_spike_corr = group['Sentiment_Score'].corr(group['Options % Spike'])

    # Aggregate score as the sum of absolute correlations
    aggregate_score = (
        abs(sentiment_spike_corr) +
        abs(sentiment_reddit_corr) +
        abs(sentiment_options_spike_corr)
    )

    correlation_results.append({
        'Company': company,
        'Sentiment_Score vs % Spike': sentiment_spike_corr,
        'Sentiment_Score vs reddit vader sentiment': sentiment_reddit_corr,
        'Sentiment_Score vs Options % Spike': sentiment_options_spike_corr,
        'Aggregate Score': aggregate_score
    })

# Create a DataFrame to store the correlation results
correlation_df = pd.DataFrame(correlation_results)

# Find top companies based on aggregate score
top_companies = correlation_df.sort_values(by='Aggregate Score', ascending=False).head(10)

# Save the correlation results
correlation_df.to_csv("company_correlations_with_aggregate_score.csv", index=False)

# Print the results
print("Top Companies by Aggregate Correlation Score:")
print(top_companies)


In [ ]:

# Ensure numerical columns are in the correct format
df['Sentiment_Score'] = pd.to_numeric(df['Sentiment_Score'], errors='coerce')
df['% Spike'] = pd.to_numeric(df['% Spike'], errors='coerce')
df['reddit vader sentiment'] = pd.to_numeric(df['reddit vader sentiment'], errors='coerce')
df['Options % Spike'] = pd.to_numeric(df['Options % Spike'], errors='coerce')

# Drop rows with missing values
df.dropna(subset=['Sentiment_Score', '% Spike', 'reddit vader sentiment', 'Options % Spike'], inplace=True)

# Group by company
company_groups = df.groupby('Company')

# Calculate correlations for each company
correlation_results = []
for company, group in company_groups:
    sentiment_spike_corr = group['Sentiment_Score'].corr(group['% Spike'])
    sentiment_reddit_corr = group['Sentiment_Score'].corr(group['reddit vader sentiment'])
    sentiment_options_spike_corr = group['Sentiment_Score'].corr(group['Options % Spike'])

    # Aggregate score as the sum of absolute correlations
    aggregate_score = (
        abs(sentiment_spike_corr) +
        abs(sentiment_reddit_corr) +
        abs(sentiment_options_spike_corr)
    )

    correlation_results.append({
        'Company': company,
        'Sentiment_Score vs % Spike': sentiment_spike_corr,
        'Sentiment_Score vs reddit vader sentiment': sentiment_reddit_corr,
        'Sentiment_Score vs Options % Spike': sentiment_options_spike_corr,
        'Aggregate Score': aggregate_score
    })

# Create a DataFrame to store the correlation results
correlation_df = pd.DataFrame(correlation_results)

# Find top companies based on aggregate score
top_companies = correlation_df.sort_values(by='Aggregate Score', ascending=False).head(10)

# Merge the top companies back with the original dataset to analyze common factors
top_company_names = top_companies['Company']
top_company_data = df[df['Company'].isin(top_company_names)]

# Analyze common factors
common_factors = top_company_data.groupby('Company').agg({
    'Sentiment_Score': ['mean', 'std'],
    '% Spike': ['mean', 'std'],
    'reddit vader sentiment': ['mean', 'std'],
    'Options % Spike': ['mean', 'std'],
    'Daily Trading Volume': 'mean',  # Assuming this column exists
    'Monthly Average Volume': 'mean',  # Assuming this column exists
})

# Display common factors
print("Common Factors for Top Companies:")
print(common_factors)

# Save common factors to a CSV file for detailed analysis
common_factors.to_csv("common_factors_for_top_companies.csv")

In [ ]:
import pandas as pd


# Add a Sector column manually or map sectors using a predefined dictionary
sector_mapping = {
    "AMERICAN TOWER CORP": "Real Estate",
    "AMGEN Inc": "Healthcare",
    "AUTOMATIC DATA PROCESSING INC": "Technology",
    "CHARTER COMMUNICATIONS INC": "Communication Services",
    "CHUBB LTD": "Financials",
    "MEDTRONIC PLC": "Healthcare",
    "MERCK & CO INC": "Healthcare",
    "NEWMONT CORP": "Materials",
    "NORTHROP GRUMMAN CORP": "Industrials",
    "VERTEX PHARMACEUTICALS INC": "Healthcare",
}
data["Sector"] = data["Company"].map(sector_mapping)

# Group by sector and calculate aggregate metrics
sector_summary = data.groupby("Sector").agg(
    {
        "Sentiment_Score": ["mean", "std"],
        "% Spike": ["mean", "std"],
        "reddit vader sentiment": ["mean", "std"],
        "Options % Spike": ["mean", "std"],
        "Daily Trading Volume": "mean",
        "Monthly Average Volume": "mean",
    }
)

print("Sector-Wise Summary:")
print(sector_summary)

# Visualize sector-wise options spike
import matplotlib.pyplot as plt

sector_options_spike = data.groupby("Sector")["Options % Spike"].mean()
sector_options_spike.plot(kind="bar", figsize=(10, 6), title="Sector-Wise Options % Spike")
plt.xlabel("Sector")
plt.ylabel("Options % Spike")
plt.show()
